# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
df = pd.read_csv("data/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.columns

Index(['Unnamed: 0', 'Ast_2', 'Ast_30', 'Ast_4', 'Baseline_points_2',
       'Baseline_points_30', 'Baseline_points_4', 'Bonus_2', 'Bonus_30',
       'Bonus_4', 'CrdR_2', 'CrdR_30', 'CrdR_4', 'CrdY_2', 'CrdY_30', 'CrdY_4',
       'DEF', 'FPL_points', 'FPL_points_2', 'FPL_points_30', 'FPL_points_4',
       'FPL_pos', 'FWD', 'GW', 'Gls_2', 'Gls_30', 'Gls_4', 'MID', 'Min_2',
       'Min_30', 'Min_4', 'Name_original', 'Opp_rating', 'Opp_score_2',
       'Opp_score_30', 'Opp_score_4', 'Opponent', 'Rating_difference',
       'Season', 'Sh_2', 'Sh_30', 'Sh_4', 'SoT_2', 'SoT_30', 'SoT_4',
       'Start_2', 'Start_30', 'Start_4', 'Team', 'Team_CS_2', 'Team_CS_30',
       'Team_CS_4', 'Team_rating', 'Team_result_2', 'Team_result_30',
       'Team_result_4', 'Team_score_2', 'Team_score_30', 'Team_score_4',
       'Was_home', 'xA_2', 'xA_30', 'xA_4', 'xG_2', 'xG_30', 'xG_4',
       'Finished', 'Date'],
      dtype='object')

In [5]:
rolling_gameweeks = [2, 4, 30]
features_to_roll = [
                "Min", "Start", # time played
                'Gls', 'Sh', 'SoT', # Goals
                'Ast', # Assists
                'CrdY', 'CrdR', # Cards
                "xG", 'xA', # Expected
                'Team_CS', # Defence
                'Team_score', 'Opp_score', 'Team_result', # Team form
                # "xGPoints", "CSPoints", # Position-scaled
                # "Cmp%", "PrgP", "PrgC", "Succ", "PKwon",
                # 'bonus', 'bps', # Bonus
                # 'influence', 'creativity', 'threat', 'ict_index', # ICT
                "FPL_points", "Baseline_points", "Bonus" # FPL points
            ]
features = ["Was_home",
            "Team_rating", "Opp_rating", "Rating_difference", 
            "DEF", "MID", "FWD"]

for r in rolling_gameweeks:
    features += [f'{col}{"_"}{r}' for col in features_to_roll]

info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos"]
to_predict = ["FPL_points"]

In [6]:
len(features)

58

In [7]:
# df = df[np.unique(features + info)]

In [8]:
file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [9]:
model

GradientBoostingRegressor(random_state=42)

In [10]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    
    preds = df_predictions[info + ["Pred"]].sort_values(by=["Pred"], ascending = False)
    
    return preds, predictions

In [11]:
# df

In [12]:
X_test = df[df["Season"] == CURRENT_SEASON][features]
# .reset_index(drop=True)

In [13]:
# X_test

In [14]:
preds, predictions = get_predictions(model, df, X_test)

15761 (15761, 68)


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [15]:
# next gameweek
preds[preds["GW"] == 1].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Pred
12924,Rico Henry,1.0,2023-24,Brentford,Tottenham,1.0,1824.330566,1815.873291,DEF,8.912238
11596,Neco Williams,1.0,2023-24,Nott'ham Forest,Arsenal,1.0,1673.962646,1919.368774,DEF,8.133387
5082,Gabriel Fernando de Jesus,1.0,2023-24,Arsenal,Nott'ham Forest,1.0,1919.368774,1673.962646,FWD,7.762316
13829,Scott McTominay,1.0,2023-24,Manchester Utd,Wolves,1.0,1873.631104,1719.037964,MID,7.743224
14661,Thomas Partey,1.0,2023-24,Arsenal,Nott'ham Forest,1.0,1919.368774,1673.962646,MID,7.714360
5158,Gabriel dos Santos Magalhães,1.0,2023-24,Arsenal,Nott'ham Forest,1.0,1919.368774,1673.962646,DEF,7.675079
5765,Ibrahima Konaté,1.0,2023-24,Liverpool,Chelsea,0.0,1946.899780,1788.094360,DEF,7.503110
8416,Kevin De Bruyne,1.0,2023-24,Manchester City,Burnley,0.0,2077.251953,1726.401489,MID,7.312964
15382,Wilfried Zaha,1.0,2023-24,Crystal Palace,Sheffield United,1.0,1757.395630,1644.073120,MID,7.290594
4628,Ethan Nwaneri,1.0,2023-24,Arsenal,Nott'ham Forest,0.0,1919.368774,1673.962646,MID,7.264195


In [16]:
# all preds
preds.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Pred
11604,Neco Williams,9.0,2023-24,Nott'ham Forest,Luton,1.0,1673.962646,1606.522583,DEF,13.060552
11624,Neco Williams,29.0,2023-24,Nott'ham Forest,Luton,1.0,1673.962646,1606.522583,DEF,13.060552
11597,Neco Williams,2.0,2023-24,Nott'ham Forest,Sheffield United,1.0,1673.962646,1644.073120,DEF,12.950106
11631,Neco Williams,36.0,2023-24,Nott'ham Forest,Sheffield United,1.0,1673.962646,1644.073120,DEF,12.950106
11613,Neco Williams,18.0,2023-24,Nott'ham Forest,Bournemouth,1.0,1673.962646,1660.637939,DEF,12.465304
11618,Neco Williams,23.0,2023-24,Nott'ham Forest,Bournemouth,1.0,1673.962646,1660.637939,DEF,12.465304
3684,Diego Da Silva Costa,6.0,2023-24,Wolves,Luton,1.0,1719.037964,1606.522583,FWD,11.748535
3713,Diego Da Silva Costa,35.0,2023-24,Wolves,Luton,1.0,1719.037964,1606.522583,FWD,11.748535
3704,Diego Da Silva Costa,26.0,2023-24,Wolves,Sheffield United,1.0,1719.037964,1644.073120,FWD,11.638088
3689,Diego Da Silva Costa,11.0,2023-24,Wolves,Sheffield United,1.0,1719.037964,1644.073120,FWD,11.638088


In [17]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Neco Williams,340.394272
Ibrahima Konaté,278.249515
Ethan Nwaneri,271.992123
Kevin De Bruyne,271.514249
Scott McTominay,268.070419
Rico Henry,265.486759
Jack Grealish,247.817827
Diego Da Silva Costa,247.337883
Harry Kane,236.917082
